In [1]:
import uuid
from typing import TypedDict, Literal, Optional
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# --- 1. SETUP QWEN (LOCAL) ---
llm = ChatOllama(model="qwen2.5:0.5b", temperature=0)

# --- 2. DEFINE STATE ---
class GraphState(TypedDict):
    topic: str              # What the email is about
    draft: str              # The current email draft
    feedback: Optional[str] # Human feedback (if any)
    status: str             # "pending", "approved", or "rejected"

# --- 3. DEFINE NODES ---

def draft_node(state: GraphState):
    """
    Writes the email. If there is feedback, it rewrites it.
    """
    topic = state["topic"]
    feedback = state.get("feedback")
    
    if feedback:
        print(f"✍️ Re-writing based on feedback: '{feedback}'")
        prompt = ChatPromptTemplate.from_template(
            "You are an email assistant. Rewrite this email draft based on the feedback.\n"
            "Original Topic: {topic}\n"
            "Feedback: {feedback}\n"
            "New Draft:"
        )
        chain = prompt | llm
        response = chain.invoke({"topic": topic, "feedback": feedback})
    else:
        print(f"✍️ Writing initial draft on: '{topic}'")
        prompt = ChatPromptTemplate.from_template(
            "Write a short, professional cold email about: {topic}.\nDraft:"
        )
        chain = prompt | llm
        response = chain.invoke({"topic": topic})
        
    return {"draft": response.content, "feedback": None} # Clear feedback after using it

def human_review_node(state: GraphState):
    """
    This node doesn't do work. It's just a placeholder where the graph resumes
    after the human interruption. It passes the state to the router.
    """
    pass # The state is already updated by the human via 'update_state'

def send_node(state: GraphState):
    print("\n🚀 EMAIL SENT! 🚀")
    print("-" * 20)
    print(state["draft"])
    print("-" * 20)
    return

# --- 4. DEFINE LOGIC (ROUTER) ---

def route_after_review(state: GraphState):
    """
    Decides where to go based on what the human said.
    """
    if state["status"] == "approved":
        return "send_node"
    else:
        return "draft_node"

# --- 5. BUILD GRAPH WITH PERSISTENCE ---

workflow = StateGraph(GraphState)

workflow.add_node("draft_node", draft_node)
workflow.add_node("human_review_node", human_review_node)
workflow.add_node("send_node", send_node)

workflow.set_entry_point("draft_node")

# Draft -> Review
workflow.add_edge("draft_node", "human_review_node")

# Review -> Router -> (Send OR Back to Draft)
workflow.add_conditional_edges(
    "human_review_node",
    route_after_review,
    {
        "send_node": "send_node",
        "draft_node": "draft_node"
    }
)

workflow.add_edge("send_node", END)

# *** CRITICAL STEP: ADD MEMORY ***
checkpointer = MemorySaver()

# *** CRITICAL STEP: CONFIGURE INTERRUPT ***
# We tell the graph: "Run until you hit 'human_review_node', then STOP and wait."
app = workflow.compile(
    checkpointer=checkpointer,
    interrupt_before=["human_review_node"]
)

# --- 6. RUN INTERACTIVE SIMULATION ---

def run_interactive_session(topic_input):
    # We need a thread_id to keep track of this specific conversation state
    thread_config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    
    print(f"🏁 Starting session for topic: {topic_input}")
    
    # Kick off the graph. It will run 'draft_node' and stop BEFORE 'human_review_node'
    app.invoke({"topic": topic_input, "status": "pending"}, config=thread_config)
    
    while True:
        # 1. Get current state (The Draft)
        # We peek into the memory to see what the AI just wrote
        current_state = app.get_state(thread_config)
        current_draft = current_state.values["draft"]
        
        print("\n" + "="*30)
        print(f"📄 CURRENT DRAFT:\n{current_draft}")
        print("="*30)
        
        # 2. Ask Human for Input
        user_input = input("User (Type 'approve' to send, or type instructions to edit): ")
        
        # 3. Update State based on Human Input
        if user_input.lower().strip() == "approve":
            # Update status to approved
            app.update_state(thread_config, {"status": "approved"})
            print("✅ Approved! Resuming graph...")
            
            # Resume graph (it enters 'human_review_node', sees 'approved', goes to 'send_node')
            app.invoke(None, config=thread_config)
            break # Exit loop
            
        else:
            # Update status to pending and inject feedback
            print("🔄 Requesting changes...")
            app.update_state(thread_config, {
                "status": "pending",
                "feedback": user_input
            })
            
            # Resume graph (it enters 'human_review_node', sees 'pending', loops back to 'draft_node')
            app.invoke(None, config=thread_config)

# --- RUN IT ---
# Try running this! It will ask for input in the console.
run_interactive_session("Selling cheap solar panels to homeowners")

🏁 Starting session for topic: Selling cheap solar panels to homeowners
✍️ Writing initial draft on: 'Selling cheap solar panels to homeowners'

📄 CURRENT DRAFT:
Subject: Selling Cheap Solar Panels to Homeowners

Dear [Recipient's Name],

I hope this message finds you well.

I am writing to discuss the potential of selling cheap solar panels to homeowners. As we move towards a more sustainable future, it is becoming increasingly important for individuals and communities to explore alternative energy sources that are both cost-effective and environmentally friendly.

Solar panels have become an increasingly popular choice among homeowners due to their high efficiency rates and long lifespan. They can be installed on rooftops or in backyard areas, making them accessible to many different types of homes.

However, it is important to note that the cost of solar panels has been decreasing over the years, making them more affordable for both individuals and businesses. This means that homeown